In [1]:
import re
import nltk
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import seaborn as sns
import dask.dataframe as dd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
ps = PorterStemmer()
import sys
sys.setrecursionlimit(5000)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
df1 = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
df2 = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")
y = df1.iloc[:,2:]
df1 = pd.DataFrame(df1.iloc[:,1])
df2 = pd.DataFrame(df2.iloc[:,1])
x=pd.DataFrame(pd.concat([df1,df2]))

In [3]:
print(df1.shape,df2.shape,x.shape)
x.head()
x.index=[a for a in range(0,len(x))]

(159571, 1) (153164, 1) (312735, 1)


In [4]:
def convert(p):
    p=re.sub("[^a-zA-Z]"," ",p)
    p=p.lower()
    p=nltk.word_tokenize(p)
    corpus=[]
    for a in p:
        if a.isalnum():
            corpus.append(a)
    corpus3=[]
    for a in corpus:
        corpus3.append(ps.stem(a))
    return " ".join(corpus3)

In [5]:
%%time
x["t_text"] = x["comment_text"].apply(convert)

CPU times: user 8min, sys: 1.1 s, total: 8min 1s
Wall time: 8min 3s


In [6]:
x.head()

,comment_text,t_text
0,Explanation\nWhy the edits made under my usern...,explan whi the edit made under my usernam hard...
1,D'aww! He matches this background colour I'm s...,d aww he match thi background colour i m seemi...
2,"Hey man, I'm really not trying to edit war. It...",hey man i m realli not tri to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",more i can t make ani real suggest on improv i...
4,"You, sir, are my hero. Any chance you remember...",you sir are my hero ani chanc you rememb what ...


In [9]:
cv = CountVectorizer(max_features=1000,ngram_range=(1,1))
dff = pd.DataFrame(cv.fit_transform(x.t_text).toarray())
dff.columns=cv.get_feature_names()

In [51]:
wo = stopwords.words("english")
for a in dff.columns:
    if a in wo:
        dff.drop(a,inplace=True,axis=1)

In [52]:
dff.shape

(312735, 892)

In [12]:
xtr = dff.iloc[:159571]
xte = dff.iloc[159571:]
xtr.shape,xte.shape

((159571, 1000), (153164, 1000))

In [14]:
#from sklearn.model_selection import train_test_split
#xtr,xte,ytr,yte = train_test_split(x,y,test_size=0.25)
y.shape

(159571, 6)

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [41]:
%%time
model1=DecisionTreeClassifier()
model1.fit(xtr,y)

CPU times: user 6min 53s, sys: 846 ms, total: 6min 53s
Wall time: 6min 55s


RandomForestClassifier()

In [ ]:
%%time
model2=KNeighborsClassifier()
model2.fit(xtr,y)

In [ ]:
%%time
model3=RandomForestClassifier()
model3.fit(xtr,y)

In [42]:
%%time
ypred1 = model1.predict(xte)

In [ ]:
%%time
ypred2 = model2.predict(xte)

In [ ]:
%%time
ypred3 = model3.predict(xte)

In [44]:
ypred1=pd.DataFrame(ypred1,columns=["toxic","severe_toxic","obscene","threat","insult","identity_hate"])
ypred2=pd.DataFrame(ypred2,columns=["toxic","severe_toxic","obscene","threat","insult","identity_hate"])
ypred3=pd.DataFrame(ypred3,columns=["toxic","severe_toxic","obscene","threat","insult","identity_hate"])

In [46]:
test1 = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip")
ypred1["id"]=test1.id
ypred1=ypred1.iloc[:,[-1,0,1,2,3,4,5]]
ypred2["id"]=test1.id
ypred2=ypred2.iloc[:,[-1,0,1,2,3,4,5]]
ypred3["id"]=test1.id
ypred3=ypred3.iloc[:,[-1,0,1,2,3,4,5]]
ypred1.to_csv("DecisionTree.csv",index=False)
ypred2.to_csv("KNN.csv",index=False)
ypred3.to_csv("RF.csv",index=False)